In [1]:
import json
from sqlite_utils import Database
from sqlite_graph import generate_adjacency_tables, in_neighborhood_cte, out_neighborhood_cte

In [2]:
with open("test_merit.json") as json_file:
    container = json.load(json_file)

In [3]:
!rm -f test_merit.db

In [4]:
db = Database('test_merit.db')

In [5]:
container["id"]

'dc30880d-2d4b-447c-905e-b79075aea95b'

In [6]:
container["fields"]

[{'canShare': False,
  'description': "Use this to store recipient's first name",
  'fieldKind': {'fieldType': 'Text',
   'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
   'name': 'Text'},
  'name': 'First Name',
  'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
  'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
  'updatedAt': '2023-07-17T15:52:19.496539Z',
  'validationErrors': None,
  'value': 'Hank'},
 {'canShare': False,
  'description': "Use this to store recipient's last name",
  'fieldKind': {'fieldType': 'Text',
   'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
   'name': 'Text'},
  'name': 'Last Name',
  'templateFieldID': '1a7fbab6-db84-49df-8196-a971afcde447',
  'templateFieldLineage': ['5029dd4c-9879-46cd-81ac-0f75ca0c46fb'],
  'updatedAt': '2023-07-17T15:52:19.496539Z',
  'validationErrors': None,
  'value': 'Lindgren'},
 {'canShare': False,
  'description': "Use this to store recipient's email address",
  'fieldKind': {'fieldType': 'Email',
   'i

<img src="example-merit-graph.png" width="1200">

In [7]:
db['vertex'].insert_all([
    { 
        "vid": container["issuer"]["id"], 
        "attributes": { 
            "type": container["issuer"]["type"], 
            "state": container["issuer"]["state"],
            } 
    },
    { 
        "vid": container["recipient"]["id"], 
        "attributes": { 
            "type": container["recipient"]["type"], 
            "state": container["recipient"]["state"],
            } 
    },
], pk='vid')

<Table vertex (vid, attributes)>

In [8]:
print(db['vertex'].schema)

CREATE TABLE [vertex] (
   [vid] TEXT PRIMARY KEY,
   [attributes] TEXT
)


In [9]:
[ row for row in db['vertex'].rows ]

[{'vid': '9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
  'attributes': '{"type": "organization", "state": "active"}'},
 {'vid': '196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'attributes': '{"type": "person", "state": "active"}'}]

In [10]:
db['edge'].insert_all([ 
    { 
        "eid": container["id"], 
        "sid": container["issuer"]["id"], 
        "tid": container["recipient"]["id"], 
        "label": container["name"], 
        "attributes": { field["name"]: field["value"] for field in container["fields"] },
        "state": container["state"]["name"],
        "container": container,
    },
], pk='eid')

<Table edge (eid, sid, tid, label, attributes, state, container)>

In [11]:
print(db['edge'].schema)

CREATE TABLE [edge] (
   [eid] TEXT PRIMARY KEY,
   [sid] TEXT,
   [tid] TEXT,
   [label] TEXT,
   [attributes] TEXT,
   [state] TEXT,
   [container] TEXT
)


In [12]:
[ row for row in db['edge'].rows ]

[{'eid': 'dc30880d-2d4b-447c-905e-b79075aea95b',
  'sid': '9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
  'tid': '196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'label': 'Admin merit',
  'attributes': '{"First Name": "Hank", "Last Name": "Lindgren", "Email": "oleg+user2@gomerits.com", "Issuing Org Name": "Merit International, Inc.", "Org UUID": "user2", "Admin Phone Number": "6317070926", "Org Legal Name": "Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd", "Org Name": "Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb", "Org Description": "This is very important organization", "Org Governing Country": "US", "Org Governing State": "CA"}',
  'state': 'accepted',
  'container': '{"id": "dc30880d-2d4b-447c-905e-b79075aea95b", "acceptedAt": "2023-07-17T08:52:19.489566Z", "active": true, "activenessFailures": null, "authorizedAt": "2023-07-17T08:52:15.960677Z", "completed": true, "completenessFailures": null, "createdAt": "2023-07-17T15:52:04.3018Z", "description": "Template for admin merit", "

In [13]:
db.execute(f'select eid, label from edge where label = "Admin merit"').fetchall()

[('dc30880d-2d4b-447c-905e-b79075aea95b', 'Admin merit')]

In [14]:
db.execute(f'select json_extract(edge.attributes, "$.First Name") from edge').fetchall()

[('Hank',)]

In [15]:
query_3 = (
    f'select eid, label, json_extract(edge.attributes, "$.Email") '
    'from edge where label = "Admin merit" and '
    'json_extract(edge.attributes, "$.First Name") = "Hank" and '
    'json_extract(edge.attributes, "$.Last Name") = "Lindgren"'
)

In [16]:
db.execute(query_3).fetchall()

[('dc30880d-2d4b-447c-905e-b79075aea95b',
  'Admin merit',
  'oleg+user2@gomerits.com')]

In [17]:
query_4 = (
    f'select tid, json_extract(edge.attributes, "$.Email"), '
    'json_extract(edge.attributes, "$.First Name"), '
    'json_extract(edge.attributes, "$.Last Name") '
    'from edge where label = "Admin merit" and '
    'json_extract(edge.attributes, "$.Email") LIKE "oleg%"'
)

In [18]:
db.execute(query_4).fetchall()

[('196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'oleg+user2@gomerits.com',
  'Hank',
  'Lindgren')]

In [19]:
generate_adjacency_tables(db)

In [20]:
print(db['outgoing'].schema)

CREATE TABLE [outgoing] (
   [vid] TEXT PRIMARY KEY,
   [label_0] TEXT,
   [edges_0] TEXT
)


In [21]:
[ row for row in db['outgoing'].rows ]

[{'vid': '9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
  'label_0': 'Admin merit',
  'edges_0': '[{"eid": "dc30880d-2d4b-447c-905e-b79075aea95b", "tid": "196a8c8f-6fd9-456b-8193-4159b8dc3ab6"}]'}]

In [22]:
print(db['incoming'].schema)

CREATE TABLE [incoming] (
   [vid] TEXT PRIMARY KEY,
   [label_0] TEXT,
   [edges_0] TEXT
)


In [23]:
[ row for row in db['incoming'].rows ]

[{'vid': '196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'label_0': 'Admin merit',
  'edges_0': '[{"eid": "dc30880d-2d4b-447c-905e-b79075aea95b", "sid": "9a5eb92a-b34a-42b9-8fb7-10ce9449e83e"}]'}]

In [24]:
out_neighborhood_cte("9a5eb92a-b34a-42b9-8fb7-10ce9449e83e", "Admin merit")

'select vid, json_extract(value, "$.eid") as eid, label_0 as label, json_extract(value, "$.tid") as tid from outgoing, json_each(outgoing.edges_0) where vid = "9a5eb92a-b34a-42b9-8fb7-10ce9449e83e" and label_0 = "Admin merit"'

In [25]:
[ row for row in db.execute(out_neighborhood_cte("9a5eb92a-b34a-42b9-8fb7-10ce9449e83e", "Admin merit")).fetchall() ]

[('9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
  'dc30880d-2d4b-447c-905e-b79075aea95b',
  'Admin merit',
  '196a8c8f-6fd9-456b-8193-4159b8dc3ab6')]

In [26]:
[ row for row in db.execute(in_neighborhood_cte("196a8c8f-6fd9-456b-8193-4159b8dc3ab6", "Admin merit")).fetchall() ]

[('196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'dc30880d-2d4b-447c-905e-b79075aea95b',
  'Admin merit',
  '9a5eb92a-b34a-42b9-8fb7-10ce9449e83e')]